<a href="https://colab.research.google.com/github/Raivotar/Kaggle_Competitions/blob/main/House_Pricing/house_price_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports e tratamento dos dados

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn import neighbors
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor


Apenas uma coluna possui valores Null

In [2]:
data = pd.read_csv("https://raw.githubusercontent.com/Raivotar/Kaggle_Competitions/main/House_Pricing/Dataset/train.csv")
data.isna().sum()

Id                 0
MSSubClass         0
MSZoning           0
LotFrontage      259
LotArea            0
                ... 
MoSold             0
YrSold             0
SaleType           0
SaleCondition      0
SalePrice          0
Length: 81, dtype: int64

In [3]:
data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,...,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,...,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,...,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,...,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,...,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


Com base na descrição das colunas, decidi selecionar somente algumas que parecem mais importantes para esse modelo inicial.

In [4]:
y = data["SalePrice"]
col = ["LotArea", "OverallQual", "OverallCond", "YearBuilt", "BsmtFinSF1", "BsmtUnfSF", "TotalBsmtSF",
       "1stFlrSF", "2ndFlrSF", "GrLivArea", "BsmtFullBath", "FullBath", "BedroomAbvGr", "GarageArea"]
X = data[col]
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=0)


# Comparativo de alguns modelos clássicos


In [5]:
tree_model = DecisionTreeRegressor(random_state=0)
tree_model.fit(train_X, train_y)
val_pred = tree_model.predict(val_X)
tree_score = mean_absolute_error(val_y, val_pred)

In [6]:
forest_model = RandomForestRegressor(random_state=0)
forest_model.fit(train_X, train_y)
val_pred = forest_model.predict(val_X)
forest_score = mean_absolute_error(val_y, val_pred)

In [7]:
gnb = GaussianNB()
gnb.fit(train_X, train_y)
val_pred = gnb.predict(val_X)
gnb_score = mean_absolute_error(val_y, val_pred)


In [8]:
knn = neighbors.KNeighborsClassifier(n_neighbors=5)
knn.fit(train_X, train_y)
val_pred = knn.predict(val_X)
knn_score = mean_absolute_error(val_y, val_pred)

In [9]:
lr = LinearRegression(normalize=True)
lr.fit(train_X, train_y)
val_pred = lr.predict(val_X)
lr_score = mean_absolute_error(val_y, val_pred)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_base.py:145: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LinearRegression())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)


  FutureWarning,


Para fazer a comparação estamos usando o erro médio absoluto


In [10]:
lista = {"tree": tree_score, "gnb": gnb_score, "knn": knn_score, "lr": lr_score, "forest": forest_score}
result = pd.Series(lista).sort_values()

Dos modelos utilizados, o que obteve o melhor desempenho sobre o data set de treino foi o Random Forest

In [11]:
result

forest    17667.590740
lr        22621.428393
tree      26139.526027
gnb       28430.216438
knn       46359.594521
dtype: float64

Aplicando os conhecimentos do curso do Kaggle sobre o XGBRregressor para fazer a mesma analise



In [12]:
xgb = XGBRegressor(random_state=0)
xgb.fit(train_X, train_y)
val_pred = xgb.predict(val_X)


[14:04:15] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [13]:
mae_1 = mean_absolute_error(val_y, val_pred)
print("Mean Absolute Error:" , mae_1)

Mean Absolute Error: 17539.37927011986


Melhorando o modelo XGB

In [14]:
xgb2 = XGBRegressor(n_estimators=1000, learning_rate=0.05)

xgb2.fit(train_X, train_y)
val_pred = xgb2.predict(val_X)
mae_2 = mean_absolute_error(val_y, val_pred)
print("Mean Absolute Error:" , mae_2)

[14:04:15] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Mean Absolute Error: 16717.187778253425


#Conclusão

Fazendo o comparativo da MAE nos modelos sem nenhum tunning, observamos que o modelo XGB apresentou o menor erro. Ao aplicar um pequeno tunning no XGB observamos uma melhora significativa no resultado.
Para melhorarmos ainda mais esses resultados:
-Poderiamos analisar e tratar melhor os dados das colunas e a acrescentando novas variáveis.
-Trabalhar o tunning dos modelos adicionando hyperparametros e fazendo novos comparativos.